In [3]:
import matplotlib
matplotlib.use('TkAgg')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [5]:
# Load the data
data = pd.read_csv('crypto-markets.csv')
btc_data = data[data['symbol'] == 'BTC'].copy()
doge_data = data[data['symbol'] == 'DOGE'].copy()
ltc_data = data[data['symbol'] == 'LTC'].copy()
btc_data.loc[:, 'date'] = pd.to_datetime(btc_data['date'])
doge_data.loc[:, 'date'] = pd.to_datetime(doge_data['date'])
ltc_data.loc[:, 'date'] = pd.to_datetime(ltc_data['date'])
btc_data.set_index('date', inplace=True)
doge_data.set_index('date', inplace=True)
ltc_data.set_index('date', inplace=True)
#missing_values = btc_data.isnull().sum() + doge_data.isnull().sum() + ltc_data.isnull().sum()
#print("Missing values:\n", missing_values)

/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(seq

In [11]:
def create_random_subsets(data, num_subsets=40, window_size=10):
    subsets = []
    num_rows = len(data)
    for _ in range(num_subsets):
        start_index = random.randint(0, num_rows - window_size)
        subset = data.iloc[start_index:start_index+window_size]
        subsets.append(subset)
    return subsets

btc_subsets = create_random_subsets(btc_data)
doge_subsets = create_random_subsets(doge_data)
ltc_subsets = create_random_subsets(ltc_data)
# print(btc_subsets)


[               slug symbol     name  ranknow    open    high     low   close  \
date                                                                           
2015-08-10  bitcoin    BTC  Bitcoin        1  265.48  267.03  262.60  264.47   
2015-08-11  bitcoin    BTC  Bitcoin        1  264.34  270.39  264.09  270.39   
2015-08-12  bitcoin    BTC  Bitcoin        1  270.60  270.67  265.47  266.38   
2015-08-13  bitcoin    BTC  Bitcoin        1  266.18  266.23  262.84  264.08   
2015-08-14  bitcoin    BTC  Bitcoin        1  264.13  267.47  261.48  265.68   
2015-08-15  bitcoin    BTC  Bitcoin        1  265.53  266.67  261.30  261.55   
2015-08-16  bitcoin    BTC  Bitcoin        1  261.87  262.44  257.04  258.51   
2015-08-17  bitcoin    BTC  Bitcoin        1  258.49  260.50  257.12  257.98   
2015-08-18  bitcoin    BTC  Bitcoin        1  257.93  257.99  211.08  211.08   
2015-08-19  bitcoin    BTC  Bitcoin        1  225.67  237.41  222.77  226.68   

                volume        market  

In [12]:
# Creating table of 10-day datasets
def create_dataset_table(subsets):
    dataset_ids = []
    dates_list = []
    close_values_list = []

    for i, subset in enumerate(subsets):
        dataset_ids.append(i + 1)

        dates = subset.index.date.tolist()  # Extract only the date part
        close_values = subset['close'].tolist()

        dates_list.append(dates)
        close_values_list.append(close_values)

    table = pd.DataFrame({
        'dataset_id': dataset_ids,
        'dates': dates_list,
        'close_values': close_values_list
    })

    return table

btc_table = create_dataset_table(btc_subsets)
doge_table = create_dataset_table(doge_subsets)
ltc_table = create_dataset_table(ltc_subsets)
# print(btc_table)


    dataset_id                                              dates  \
0            1  [2015-08-10, 2015-08-11, 2015-08-12, 2015-08-1...   
1            2  [2017-09-18, 2017-09-19, 2017-09-20, 2017-09-2...   
2            3  [2014-06-22, 2014-06-23, 2014-06-24, 2014-06-2...   
3            4  [2017-12-12, 2017-12-13, 2017-12-14, 2017-12-1...   
4            5  [2016-06-09, 2016-06-10, 2016-06-11, 2016-06-1...   
5            6  [2018-03-09, 2018-03-10, 2018-03-11, 2018-03-1...   
6            7  [2013-09-26, 2013-09-27, 2013-09-28, 2013-09-2...   
7            8  [2018-02-08, 2018-02-09, 2018-02-10, 2018-02-1...   
8            9  [2015-05-01, 2015-05-02, 2015-05-03, 2015-05-0...   
9           10  [2017-01-22, 2017-01-23, 2017-01-24, 2017-01-2...   
10          11  [2014-08-14, 2014-08-15, 2014-08-16, 2014-08-1...   
11          12  [2015-05-22, 2015-05-23, 2015-05-24, 2015-05-2...   
12          13  [2014-07-20, 2014-07-21, 2014-07-22, 2014-07-2...   
13          14  [2016-03-10, 2016-

In [9]:
# Adding label collumn containing information about rise/ fall
def add_label_column(table):
    for index, row in table.iterrows():
        close_day_7 = row['close_values'][6]
        close_day_10 = row['close_values'][9]

        label = int(close_day_10 > close_day_7)

        table.at[index, 'rise'] = label

    return table

btc_table_with_label = add_label_column(btc_table)
ltc_table_with_label = add_label_column(ltc_table)
doge_table_with_label = add_label_column(doge_table)
# print(btc_table_with_label)

    dataset_id                                              dates  \
0            1  [2016-09-20, 2016-09-21, 2016-09-22, 2016-09-2...   
1            2  [2017-11-01, 2017-11-02, 2017-11-03, 2017-11-0...   
2            3  [2015-06-27, 2015-06-28, 2015-06-29, 2015-06-3...   
3            4  [2014-05-26, 2014-05-27, 2014-05-28, 2014-05-2...   
4            5  [2017-01-17, 2017-01-18, 2017-01-19, 2017-01-2...   
5            6  [2016-08-01, 2016-08-02, 2016-08-03, 2016-08-0...   
6            7  [2017-06-07, 2017-06-08, 2017-06-09, 2017-06-1...   
7            8  [2016-08-22, 2016-08-23, 2016-08-24, 2016-08-2...   
8            9  [2015-03-18, 2015-03-19, 2015-03-20, 2015-03-2...   
9           10  [2016-05-03, 2016-05-04, 2016-05-05, 2016-05-0...   
10          11  [2018-03-21, 2018-03-22, 2018-03-23, 2018-03-2...   
11          12  [2015-07-05, 2015-07-06, 2015-07-07, 2015-07-0...   
12          13  [2018-04-04, 2018-04-05, 2018-04-06, 2018-04-0...   
13          14  [2017-09-03, 2017-